In [0]:
%sql
CREATE STREAMING LIVE TABLE customers 
COMMENT 'This table is for customer data' 
AS SELECT *,current_timestamp() as ingested_date FROM cloud_files(
    "dbfs:/mnt/rawdatastoragefiles/Customer/",
    "csv",
    map("header", "true", "inferSchema", "true")
);

In [0]:
CREATE STREAMING LIVE TABLE product 
COMMENT 'This table is for product data' 
AS SELECT *,current_timestamp() as ingested_date FROM cloud_files(
    "dbfs:/mnt/rawdatastoragefiles/Product/",
    "csv",
    map("header", "true", "inferSchema", "true")
);

In [0]:
CREATE STREAMING LIVE TABLE sales 
COMMENT 'This table is for Sales data' 
AS SELECT *,current_timestamp() as ingested_date FROM cloud_files(
    "dbfs:/mnt/rawdatastoragefiles/Sales/",
    "csv",
    map("header", "true", "inferSchema", "true")
);

In [0]:
create streaming table sales_silver
(
  constraint valid_id expect (order_id is not null) on violation drop row
) 
as select distinct * from stream(live.sales);

In [0]:
CREATE OR REFRESH STREAMING TABLE customer_silver;

APPLY CHANGES INTO
  customer_silver
FROM
  stream(live.customers)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (_rescued_data, sequenceNum,ingested_date)
STORED AS
  SCD TYPE 2;

In [0]:
create or replace streaming live table products_silver;

apply changes into products_silver
from stream(live.product)
keys(product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
sequence by 
seqNum
columns * except(_rescued_data, seqNum,ingested_date)
stored as 
SCD type 1;


In [0]:
create or replace streaming table customers_silver_active 
as select * except(operation,__END_AT,__START_AT) from stream(live.customer_silver) where __END_AT IS NULL;

In [0]:
create or replace live table total_sales_customers_wise
as select a.customer_id,
a.customer_name,
round(sum(b.total_amount)) as total_sales,
round(sum(b.discount_amount)) as total_discounts
 from live.customers_silver_active a 
join live.sales_silver b on a.customer_id=b.customer_id
group by a.customer_id,a.customer_name
order by total_sales desc;

In [0]:
create or replace live table products_category_sales
as 
select a.product_category
,round(sum(b.total_amount)) as total_sales
from live.products_silver a 
join live.sales_silver b on a.product_id=b.product_id
group by a.product_category
order by total_sales desc;